In [2]:
.libPaths()

[1] "/home/jpm73279/.conda/envs/R_final_install/lib/R/library"

In [3]:
library(tidyverse)

ERROR: Error in library(tidyverse): there is no package called ‘tidyverse’


In [2]:
pathways <- read_delim("/home/jpm73279/genome_downloads/Zm-B73-REFERENCE-NAM-5.0/Zm-B73-REFERENCE-NAM.UniProt.GO.tab", delim = "\t" ,
                      col_names = c("geneID", "term", "GOID"))

Rows: 102169 Columns: 3
── Column specification ───────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): geneID, term, GOID

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
Sb_Zm_Ortho <- read_delim("/scratch/jpm73279/13.ACR_CNS_analysis/01.analysis/run_orthofinder/2022-06-21_plant_genomes_grass_lineages/OrthoFinder/Results_Jun22/Orthologues/Orthologues_Sbicolor.prot/Sbicolor.prot__v__Zm-B73.prot.tsv",
          delim = "\t" , col_names = c("orthogroups", "Sb_genes", "Zm_genes"))

Rows: 20082 Columns: 3
── Column specification ───────────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): orthogroups, Sb_genes, Zm_genes

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
sb_regex <- ".[^.]+$"

final_sb_zm_pairings <- separate_rows(Sb_Zm_Ortho, Sb_genes, sep = ",")  %>% 
    separate_rows(., Zm_genes, sep = ",")  %>%
    mutate(real_zm_name = str_replace(Zm_genes, "[_P].*", ""))  %>% 
    mutate(real_sb_name = str_replace(Sb_genes, sb_regex, ""))  %>% 
    dplyr::select(real_zm_name, real_sb_name)
    

In [5]:
pathways

geneID,term,GOID
<chr>,<chr>,<chr>
Zm00001eb007090,commitment complex,GO:0000243
Zm00001eb007090,U1 snRNP,GO:0005685
Zm00001eb007090,U2-type prespliceosome,GO:0071004
Zm00001eb007090,mRNA 5'-splice site recognition,GO:0000395
Zm00001eb007090,RNA processing,GO:0006396
Zm00001eb007090,RNA processing,GO:0006396
Zm00001eb007090,RNA processing,GO:0006396
Zm00001eb007090,RNA processing,GO:0006396
Zm00001eb007090,RNA processing,GO:0006396


In [7]:
cols_to_check <- c("real_zm_name", "real_sb_name","term","GOID")
final <- left_join(final_sb_zm_pairings, pathways, by = c("real_zm_name" = "geneID"))  %>% 
    dplyr::filter(if_all(cols_to_check, ~ !is.na(.x)))

[1] 74876     4

In [8]:
sb_final_go_terms <- final  %>% 
    dplyr::select(real_sb_name, term, GOID)

In [17]:
sb_test_go <- sb_final_go_terms  %>% 
    dplyr::select(real_sb_name, term)  %>% 
    group_by(term)  %>% 
    nest()  %>% 
    unnest

Warning message:
“`cols` is now required when using unnest().
Please use `cols = c(data)`”


In [18]:
head(sb_test_go)

term,real_sb_name
<chr>,<chr>
response to auxin,Sobic.002G285200
response to auxin,Sobic.002G285200
response to auxin,Sorbiv5.1_pg24730
response to auxin,Sorbiv5.1_pg24730
response to auxin,Sobic.001G184400
response to auxin,Sobic.001G184400


In [13]:
write_delim(sb_test_go, "/home/jpm73279/genome_downloads/Sbicolor.v5.1.prelim.annot/Sbicolorv5.1.GO_terms_from_ZM.collapsed.txt", delim = "\t")

In [46]:
write_delim(sb_final_go_terms, "/home/jpm73279/genome_downloads/Sbicolor.v5.1.prelim.annot/Sbicolorv5.1.GO_terms_from_ZM.txt", delim = "\t")